# **Libraries**

In [ ]:
!pip install QuantLib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 100.7 MB/s eta 0:00:00


In [1]:
import QuantLib as ql
import numpy as np
import pandas as pd
import scipy as sc
import openpyxl
import codecs
import copy
import os
import time
from datetime import datetime
from contextlib import closing
import sqlite3
from scipy.interpolate import interp1d
from contextlib import closing


ModuleNotFoundError: No module named 'QuantLib'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# **Data parsing**

In [ ]:
# Подключение Google Drive
from google.colab import drive # Тут необходимо подлючится к диску через Google аккаунт
drive.mount('/content/drive', force_remount=False) # В одной сессии хватит подключится один раз, дальше подлючение не требуется (только при перезапуске сессии)

Mounted at /content/drive


In [ ]:
# Настройка путей
db_name = 'Data Archive.db'
local_db_path = '/content/' + db_name
drive_db_path = '/content/drive/MyDrive/Data Base/' + db_name  # Путь к базе данных в Google Drive
# excel_path = '/content/drive/My Drive/Data Base/Data Archive.xlsm'  # Путь к Excel файлу

In [ ]:
# Копируем базу из Drive если она существует
if os.path.exists(drive_db_path):
  !cp "{drive_db_path}" "{local_db_path}"
  print("База данных загружена из Google Drive")
else:
  print(f"База данных {db_name} отсутствует в Google Drive")

База данных загружена из Google Drive


In [ ]:
con = sqlite3.connect('Data Archive.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("\nTables:")
print("-------")
for table in tables:
    print(table[0])


for table in tables:
    table_name = table[0]
    print(f"\nТаблица: {table_name}")
    df = pd.read_sql(f"SELECT * FROM {table_name}", con)
    print(df)



# fx_curve = pd.read_sql("SELECT DISTINCT source FROM 'fx_rate'", con)
# print(fx_curve)


con.close()


Tables:
-------
curve_quote
fx_rate
swap_points_fwd

Таблица: curve_quote
                        date           curve   source currency currency_csa  \
0        2023-07-01 00:00:00   RUB_RUSFAR_3M     MOEX      RUB          RUB   
1        2023-07-01 00:00:00   RUB_RUSFAR_3M     MOEX      RUB          RUB   
2        2023-07-01 00:00:00   RUB_RUSFAR_3M     MOEX      RUB          RUB   
3        2023-07-01 00:00:00   RUB_RUSFAR_3M     MOEX      RUB          RUB   
4        2023-07-01 00:00:00   RUB_RUSFAR_3M     MOEX      RUB          RUB   
...                      ...             ...      ...      ...          ...   
2670122  2025-07-31 00:00:00  JPY TONAR TKFX  Foreign      JPY          JPY   
2670123  2025-07-31 00:00:00  JPY TONAR TKFX  Foreign      JPY          JPY   
2670124  2025-07-31 00:00:00  JPY TONAR TKFX  Foreign      JPY          JPY   
2670125  2025-07-31 00:00:00  JPY TONAR TKFX  Foreign      JPY          JPY   
2670126  2025-07-31 00:00:00  JPY TONAR TKFX  Foreign   

# **Архитектура DataArchive по таблицам (доступные источники, курсы)**

**fx_rate**

источники и доступные столбцы в них:

1.   Cbonds - px_last
2.   CBR - px_clir
3.   Foreign - px_low, px_high, px_close
4.   NCC - px_last
5.   OFFSHORE - в актуальной версии бд данных по источнику нет ни в одном из столбцов
6.   ONSHORE - px_clir
7.   RuData FOREX -  в актуальной версии бд данных по источнику нет ни в одном из столбцов
8.   RuData MOEX - px_low, px_high, px_last

**curve_quote**

источники:

1.   Cbonds - есть пропуски в данных
2.   Foreign - все данны есть
3.   RuData - тоже все ок

Для всех источников доступный столбец со значениями - zero_rate


Сейчас ставки подгружаются по фильтрам источника, валюты, тенора/дней, ставка не прописывается явно

**swap_points_fwd**

источники:

1.   Cbonds - last (единственное значение, не подходит для диапазона min/max)
2.   MOEX - low, high, last
3.   MOEX Backtrader - low, high

In [ ]:
# Если данные не парсятся и выдают ошибку по типу index out of list, то дописать в pd.to_datetime (value, format = '%Y-%m-%d')

# **Reestr parsing**

In [ ]:
reestr_initial = pd.read_excel('reestr_new_1.xlsx')
reestr_initial.head()


,№ сделки,№ сделки (идентификатор сделки),"Статус сделки (новая, пролонгация, изменение условий, закрыта и др.)","Дата сделки, дата пролонгации/ изменения условий",Контрагент,Продаваемая валюта1,Сумма продажи1,Покупаемая валюта1,Сумма покупки1,Курс1,Дата платежа1,Продаваемая валюта2,Сумма продажи2,Покупаемая валюта2,Сумма покупки2,Курс2,Дата платежа2,"Признак продления, внесения изменений (да/нет)","Справедливая стоимость на текущую дату, в тыс. руб. (Только для ПФИ)","Текущая дата, на которую рассчитывалась справедливая стоимость ПФИ"
0,1,NaN,Новая,31.07.2024,NaN,RUB,162335547,CNY,13700000,11.84931,31.07.2024,CNY,13700000,RUB,162379250,11.8525,01.08.2024,NaN,215.09,01.08.2024
1,2,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,31.07.2024,CNY,100000,RUB,1185250,11.8525,01.08.2024,NaN,1.57,01.08.2024
2,3,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,31.07.2024,CNY,100000,RUB,1185250,11.8525,01.08.2024,NaN,1.57,01.08.2024
3,4,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,31.07.2024,CNY,100000,RUB,1185250,11.8525,01.08.2024,NaN,1.57,01.08.2024
4,5,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,31.07.2024,CNY,100000,RUB,1185250,11.8525,01.08.2024,NaN,1.57,01.08.2024


# **FUNCTIONS**


In [ ]:
# Словарь приоритезации валют
currency_priority_dict = {
    'XDR': 0,
    'EUR': 1,
    'GBP': 2,
    'AUD': 3,
    'NZD': 4,
    'USD': 5,
    'CAD': 6,
    'CHF': 7,
    'TRY': 8,
    'DKK': 9,
    'NOK': 10,
    'ZAR': 11,
    'SEK': 12,
    'CNY': 13,
    'CNH': 14,
    'BRL': 15,
    'HKD': 16,
    'INR': 17,
    'CZK': 18,
    'KZT': 19,
    'JPY': 20,
    'BYN': 21,
    'SGD': 22,
    'KRW': 23,
    'HUF': 24,
    'RON': 25,
    'AZN': 26,
    'BGN': 27,
    'UZS': 28,
    'KGS': 29,
    'MDL': 30,
    'PLN': 31,
    'TMT': 32,
    'TJS': 33,
    'UAH': 34,
    'AMD': 35,
    'GEL': 36,
    'RUB': 37
}


# находим разницу по своп пунктам через ставки
# def swap_points_simple(spot, internal_rate, external_rate, time):
#   simple = (spot*(1+(internal_rate)*time/365)/(1+(external_rate)*time/365))-spot
#   return simple



In [ ]:
# Функция определения валютной пары
def currency_pair(curr1, curr2):

    try:
        if currency_priority_dict[curr1] < currency_priority_dict[curr2]:
            pair = curr1 + curr2
        else:
            pair = curr2 + curr1
        return pair
    except KeyError:
        print("Валюта не найдена в currency_priority_dict! (Словарь приоритезации валют)")

In [ ]:
# Функция определения направления сделки
def deals_dir(curr1, curr2, curr_pair):
    """
    Input: currency 1, currency 2 and currency pair as str
    Output: direction of deal according to input parametres (SELL OR BUY)
    """
    if (curr1 in curr_pair) and (curr2 in curr_pair):
        if curr_pair[:3] == curr1:
            ddir = 'SELL'
        elif curr_pair[:3] == curr2:
            ddir = 'BUY'
    else:
        print('Несоответствие валют. Проверьте входные данные!')
        ddir = None
    return ddir

In [ ]:
# Функция определения суммы базовой валюты
def sum_curr_base(curr1, curr2, curr_pair, sum_sell, sum_buy):
    """
    Input: currency 1, currency 2 and currency pair as str; sum sell and sum buy
    Output: sum of base currency
    """
    if (curr1 in curr_pair) and (curr2 in curr_pair):
        if curr_pair[:3] == curr1:
            sum_base = sum_sell
        elif curr_pair[:3] == curr2:
            sum_base = sum_buy
    else:
        print('Несоответствие валют. Проверьте входные данные!')
        sum_base = None
    return sum_base

In [ ]:
# Функция определение суммы расчетной валюты
def sum_curr_calc(curr1, curr2, curr_pair, sum_sell, sum_buy):
    """
    Input: currency 1, currency 2 and currency pair as str; sum sell and sum buy
    Output: sum of calculation currency
    """
    if (curr1 in curr_pair) and (curr2 in curr_pair):
        if curr_pair[:3] == curr2:
            sum_curr = sum_sell
        elif curr_pair[:3] == curr1:
            sum_curr = sum_buy
    else:
        print('Несоответствие валют. Проверьте входные данные!')
        sum_curr = None
    return sum_curr

In [ ]:
# Функция определения расхождения спот курсов
def diff_spot(spot_rate, spot_min_rate, spot_max_rate):
    """
    Input: spot_rate, spot_min_rate and spot_max_rate as float
    Output: spot max exchange rate on the spot_date from the specified source
    """
    if (spot_min_rate == None) or (spot_min_rate == 0) \
        or (spot_max_rate == None) or (spot_max_rate == 0):
        diff_spot = 'Нет данных'
    else:
        if (spot_rate >= spot_min_rate) and (spot_rate <= spot_max_rate):
            diff_spot = 'В рынке'
        elif spot_rate < spot_min_rate:
            diff_spot = (spot_rate - spot_min_rate) / spot_min_rate
        else:
            diff_spot = (spot_rate - spot_max_rate) / spot_max_rate
    return diff_spot

In [ ]:
# Функция определения расхождения своп-разницы
def diff_swap_points(points_bank, points_min, points_max):
    """
    Input: swap points from bank and calculated min and max as float
    Output: fwd rate as float in a simple compounding type
    """
    if points_min in (0, None, 'Нет данных') or points_max in (0, None, 'Нет данных'):
        print('NO data')
        #diff_swap = points_bank # поч при отсутствии данных по поинтам они принимаются равными банковским данным ???______________________________________________________
    elif (abs((points_min + points_max)/2 - points_bank) <= \
          (max(points_min, points_max) - min(points_min, points_max))/2):
        diff_swap = 'В рынке'
    elif points_bank < min(points_min, points_max):
        diff_swap = points_bank - min(points_min, points_max)
    else:
        diff_swap = points_bank - max(points_min, points_max)
    return diff_swap

In [ ]:
# INITIAL FUNCTION FROM PYTHON_SWAP_BETA_1.6

# Функция определения расхождения своп-разницы
def diff_swap_points(points_bank, points_min, points_max):
    """
    Input: swap points from bank and calculated min and max as float
    Output: fwd rate as float in a simple compounding type
    """
    if (abs((points_min + points_max)/2 - points_bank) <= \
          (max(points_min, points_max) - min(points_min, points_max))/2):
        diff_swap = 'В рынке'
    elif points_bank < min(points_min, points_max):
        diff_swap = points_bank - min(points_min, points_max)
    else:
        diff_swap = points_bank - max(points_min, points_max)
    return diff_swap


In [ ]:
# Функция определения источника своп поинтов (напрямую с рынка для сделок < 3 дней или через ставки)
def type_swp(duration_of_contract):
  if duration_of_contract <= 3:
    type_swp = 'direct swp'
  else:
    type_swp = 'rates'
  return type_swp

In [ ]:
#!!!!!!!
# Из-за того, что у MOEX не было тенора в 1 день, используем пока 7 для проверки корректной работы скрипта, но при переходе на улучшенную версию базы поменять 7 на 1
#!!!!!!!

# определяем по сроку сделки больший тенор для ставок
def tenor1h(days_of_swap):

  if days_of_swap < 7:
    tenor = 7
#   if days_of_swap <=7 and days_of_swap >=14: #когда будет 1 тенор
#     tenor = 7
  if days_of_swap <=14 and days_of_swap >=7:
    tenor = 14
  if days_of_swap <=30 and days_of_swap >=14:
    tenor = 30
  if days_of_swap <= 60 and days_of_swap >=30:
    tenor = 60
  if days_of_swap <= 90 and days_of_swap >=60:
    tenor = 90
  if days_of_swap <= 180 and days_of_swap >=90:
    tenor = 180
  if days_of_swap <= 270 and days_of_swap >=180:
    tenor = 270
  if days_of_swap <= 360 and days_of_swap >=270:
    tenor = 360
  return tenor

# определяем по сроку сделки меньший тенор для ставок
def tenor2l(days_of_swap):
  if days_of_swap < 7:
    tenor = 7
  if days_of_swap <=14 and days_of_swap >=7:
    tenor = 7
  if days_of_swap <=30 and days_of_swap >=14:
    tenor = 14
  if days_of_swap <= 60 and days_of_swap >=30:
    tenor = 30
  if days_of_swap <= 90 and days_of_swap >=60:
    tenor = 60
  if days_of_swap <= 180 and days_of_swap >=90:
    tenor = 90
  if days_of_swap <= 270 and days_of_swap >=180:
    tenor = 180
  if days_of_swap <= 360 and days_of_swap >= 270:
    tenor = 270
  return tenor

In [ ]:
# задаем кривые для валют
curves = {
    'CNY': 'CNY_CNY',
    'USD': 'USD SOFR REF',
    'EUR': 'EUR ESTR REF',
    'AUD': 'AUD AONIA ICAA',
    'CAD': 'CAD CORRA REF',
    'CHF': 'CHF SARON ICAP',
    'GBP': 'GBP SONIA TWMK',
    'HKD': 'HKD HONIA TRHK',
    'JPY': 'JPY TONAR TKFX',
    'RUB': 'RUB_RUB',
    'TRY': 'TRY TLREF REF'
    }


def curve_ident(currency_1):
        x = curves.get(currency_1)
        return x

In [ ]:
#Eли брать ставки с рефа, то там есть овернайт по CNY, USD, EUR

In [ ]:
#находим внутренюю ставку меньшую
def rate_domestic_pars_l(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'MOEX'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_d_l = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}' and curve = '{curve}' """)).fetchall()[0][0])
  return rate_d_l

#находим внутренюю ставку большую
def rate_domestic_pars_h(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'MOEX'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_d_h = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}' and curve = '{curve}' """)).fetchall()[0][0])
  return rate_d_h

#находим внешнюю ставку меньшую
def rate_foreign_pars_l(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'MOEX'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_f_l = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}'and curve = '{curve}' """)).fetchall()[0][0])
  return rate_f_l

#находим внешнюю ставку большую
def rate_foreign_pars_h(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'MOEX'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_f_h = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}' and curve = '{curve}' """)).fetchall()[0][0])
  return rate_f_h

# Блок для проверки рыночных данных на наличие (все функции уже к этому моменту определены, прежде чем вытаскивать в окончательный реестр, сделаем блок с проверкой)

In [ ]:
def exist(date, table, currency_pair, source):
    """
    Input: date, name of table from database, currency pair for parsing, source (according to table)
    """
    date = pd.to_datetime(date, dayfirst = True)

    try:
        with closing(sqlite3.connect('Data Archive.db')) as connection:
            ncc = ((connection.cursor().execute(f"""select px_last from '{table_name}' where source = '{source}' and pair = '{pair}' and date = '{date}' """)).fetchall()[0][0])
    except KeyError:
        print("Данных по валюте нет")
    return ncc

In [ ]:
reestr_mini = pd.read_excel('reestr_new_1.xlsx')
reestr_mini

,№ сделки,№ сделки (идентификатор сделки),"Статус сделки (новая, пролонгация, изменение условий, закрыта и др.)","Дата сделки, дата пролонгации/ изменения условий",Контрагент,Продаваемая валюта1,Сумма продажи1,Покупаемая валюта1,Сумма покупки1,Курс1,Дата платежа1,Продаваемая валюта2,Сумма продажи2,Покупаемая валюта2,Сумма покупки2,Курс2,Дата платежа2,"Признак продления, внесения изменений (да/нет)","Справедливая стоимость на текущую дату, в тыс. руб. (Только для ПФИ)","Текущая дата, на которую рассчитывалась справедливая стоимость ПФИ"
0,1,NaN,Новая,31.07.2024,NaN,RUB,162335547,CNY,13700000,11.84931,31.07.2024,CNY,13700000,RUB,162379250,11.85250,01.08.2024,NaN,215.09,01.08.2024
1,2,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,31.07.2024,CNY,100000,RUB,1185250,11.85250,01.08.2024,NaN,1.57,01.08.2024
2,3,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,31.07.2024,CNY,100000,RUB,1185250,11.85250,01.08.2024,NaN,1.57,01.08.2024
3,4,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,31.07.2024,CNY,100000,RUB,1185250,11.85250,01.08.2024,NaN,1.57,01.08.2024
4,5,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,31.07.2024,CNY,100000,RUB,1185250,11.85250,01.08.2024,NaN,1.57,01.08.2024
5,6,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,31.07.2024,CNY,100000,RUB,1185250,11.85250,01.08.2024,NaN,1.57,01.08.2024
6,7,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,31.07.2024,CNY,100000,RUB,1185250,11.85250,01.08.2024,NaN,1.57,01.08.2024
7,8,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,31.07.2024,CNY,100000,RUB,1185250,11.85250,01.08.2024,NaN,1.57,01.08.2024
8,9,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,31.07.2024,CNY,100000,RUB,1185250,11.85250,01.08.2024,NaN,1.57,01.08.2024
9,10,NaN,Новая,31.07.2024,NaN,RUB,2369862,CNY,200000,11.84931,31.07.2024,CNY,200000,RUB,2370500,11.85250,01.08.2024,NaN,3.14,01.08.2024


# **CALCULATE REESTR**

**The most actual version**

In [ ]:
reestr_initial = pd.read_excel('reestr_new_1.xlsx')

In [ ]:
# Загружаем реестр, определяем исходные параметры для расчетов и валюту расчетов (RUB)
reestr_mini = pd.read_excel('reestr_new_1.xlsx')


reestr_mini['Дата оценки'] = reestr_mini['Текущая дата, на которую рассчитывалась справедливая стоимость ПФИ']
reestr_mini['Продаваемая Банком валюта'] = reestr_mini['Продаваемая валюта1']
reestr_mini['Покупаемая Банком валюта'] = reestr_mini['Покупаемая валюта1 ']
reestr_mini['Валюта расчетов'] = input("Введите валюту расчетов (например: RUB): ")


Введите валюту расчетов (например: RUB): RUB


In [ ]:
# Вспомогательная часть для валютных пар
reestr_mini['Валютная пара 1'] = reestr_mini['Покупаемая валюта2 '] + reestr_mini['Валюта расчетов']
reestr_mini['Валютная пара 2'] = reestr_mini['Покупаемая валюта1 '] + reestr_mini['Валюта расчетов']

In [ ]:
# Валютная пара
curr_pair = []
for i in range(len(reestr_mini['№ сделки'])):
    curr_pair.append(currency_pair(reestr_mini['Продаваемая Банком валюта'].iloc[i], reestr_mini['Покупаемая Банком валюта'].iloc[i]))
reestr_mini['Валютная пара'] = curr_pair

In [ ]:
# Направление сделки
direction = []
for i in range(len(reestr_mini['№ сделки'])):
    direction.append(deals_dir(reestr_mini['Продаваемая Банком валюта'].iloc[i], reestr_mini['Покупаемая Банком валюта'].iloc[i], reestr_mini['Валютная пара'].iloc[i]))
reestr_mini['Направление сделки'] = direction
reestr_mini['Название'] = reestr_mini['Валютная пара'] + reestr_mini['Направление сделки']

In [ ]:
# Сумма в базовой валюте
sum_base = []
for i in range(len(reestr_mini['№ сделки'])):
    sum_base.append(sum_curr_base(reestr_mini['Продаваемая Банком валюта'].iloc[i], reestr_mini['Покупаемая Банком валюта'].iloc[i], reestr_mini['Валютная пара'].iloc[i], reestr_mini['Сумма продажи1'].iloc[i], reestr_mini['Сумма покупки1'].iloc[i]))
reestr_mini['Сумма в базовой валюте'] = sum_base

In [ ]:
# Сумма в расчетной валюте
sum_calc = []
for i in range(len(reestr_mini['№ сделки'])):
    sum_calc.append(sum_curr_calc(reestr_mini['Продаваемая Банком валюта'].iloc[i], reestr_mini['Покупаемая Банком валюта'].iloc[i], reestr_mini['Валютная пара'].iloc[i], reestr_mini['Сумма продажи1'].iloc[i], reestr_mini['Сумма покупки1'].iloc[i]))
reestr_mini['Сумма в расчетной валюте'] = sum_calc

reestr_mini['Курс спот сделки (1 части)'] = reestr_mini['Курс1']

In [ ]:
# ЦК НКЦ
ncc_rate = []
for i in range(len(reestr_mini['№ сделки'])):
    pair_NCC_name = reestr_mini['Валютная пара'].iloc[i]
    date = pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/ изменения условий'].iloc[i], dayfirst=True)
    table_name = 'fx_rate'
    source = 'NCC'
    with closing(sqlite3.connect('Data Archive.db')) as connection:
        result = connection.cursor().execute(f"""select px_last from '{table_name}' where source = '{source}' and pair = '{pair_NCC_name}' and date = '{date}' """).fetchall()
        ncc_rate.append(result[0][0] if result else None)
reestr_mini['ЦК НКЦ'] = ncc_rate


In [ ]:
# Курсы для СС ЦК НКЦ
ncc_1 = []
for i in range(len(reestr_mini['№ сделки'])):
    if reestr_mini['Валютная пара 1'].iloc[i] == 'RUBRUB':
        ncc_1.append(1)
    else:
        ncc_1.append(reestr_mini['ЦК НКЦ'].iloc[i])
reestr_mini['ncc_1'] = ncc_1

ncc_2 = []
for i in range(len(reestr_mini['№ сделки'])):
    if reestr_mini['Валютная пара 2'].iloc[i] == 'RUBRUB':
        ncc_2.append(1)
    else:
        ncc_2.append(reestr_mini['ЦК НКЦ'].iloc[i])
reestr_mini['ncc_2'] = ncc_2

In [ ]:
# Спот мин на дату сделки
spot_min = []
for i in range(len(reestr_mini['№ сделки'])):
    table_name = 'fx_rate'
    source = 'Foreign'
    pair_spot_name = reestr_mini['Валютная пара'].iloc[i]
    date = pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/ изменения условий'].iloc[i], dayfirst=True)
    with closing(sqlite3.connect('Data Archive.db')) as connection:
        result = connection.cursor().execute(f"""select px_low from '{table_name}' where source = '{source}' and pair = '{pair_spot_name}' and date = '{date}' """).fetchall()
        spot_min.append(result[0][0] if result else None)
reestr_mini['Спот мин'] = spot_min

In [ ]:
# Спот мин на дату оценки
spot_min_val_date = []
for i in range(len(reestr_mini['№ сделки'])):
    table_name = 'fx_rate'
    source = 'Foreign'
    pair_spot_name = reestr_mini['Валютная пара'].iloc[i]
    date = pd.to_datetime(reestr_mini['Текущая дата, на которую рассчитывалась справедливая стоимость ПФИ'].iloc[i], dayfirst=True)
    with closing(sqlite3.connect('Data Archive.db')) as connection:
        result = connection.cursor().execute(f"""select px_low from '{table_name}' where source = '{source}' and pair = '{pair_spot_name}' and date = '{date}' """).fetchall()
        spot_min_val_date.append(result[0][0] if result else None)
reestr_mini['Спот мин val date'] = spot_min_val_date

In [ ]:
# Спот макс на дату сделки
spot_max = []
for i in range(len(reestr_mini['№ сделки'])):
    table_name = 'fx_rate'
    source = 'Foreign'
    pair_spot_name = reestr_mini['Валютная пара'].iloc[i]
    date = pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/ изменения условий'].iloc[i], dayfirst=True)
    with closing(sqlite3.connect('Data Archive.db')) as connection:
        result = connection.cursor().execute(f"""select px_high from '{table_name}' where source = '{source}' and pair = '{pair_spot_name}' and date = '{date}' """).fetchall()
        spot_max.append(result[0][0] if result else None)
reestr_mini['Спот макс'] = spot_max

In [ ]:
# Спот макс на дату оценки
spot_max_val_date = []
for i in range(len(reestr_mini['№ сделки'])):
    table_name = 'fx_rate'
    source = 'Foreign'
    pair_spot_name = reestr_mini['Валютная пара'].iloc[i]
    date = pd.to_datetime(reestr_mini['Текущая дата, на которую рассчитывалась справедливая стоимость ПФИ'].iloc[i], dayfirst=True)
    with closing(sqlite3.connect('Data Archive.db')) as connection:
        result = connection.cursor().execute(f"""select px_high from '{table_name}' where source = '{source}' and pair = '{pair_spot_name}' and date = '{date}' """).fetchall()
        spot_max_val_date.append(result[0][0] if result else None)
reestr_mini['Спот макс val date'] = spot_max_val_date

In [ ]:
# Разница спот
spot_diff = []
for i in range(len(reestr_mini['№ сделки'])):
    spot_diff.append(diff_spot(reestr_mini['Курс спот сделки (1 части)'].iloc[i], reestr_mini['Спот мин'].iloc[i], reestr_mini['Спот макс'].iloc[i]))
reestr_mini['Расхождение'] = spot_diff

In [ ]:
# Срок, дней
days = []
for i in range(len(reestr_mini['№ сделки'])):
    a = pd.to_datetime(reestr_mini['Дата платежа2'].iloc[i], dayfirst=True) - pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/ изменения условий'].iloc[i], dayfirst=True)
    days.append(a.days)
reestr_mini['Срок'] = days

In [ ]:
# Тип своп-поинтов (ставки/рынок)
swap_type = []
for i in range(len(reestr_mini['№ сделки'])):
    swap_type.append(type_swp(reestr_mini['Срок'].iloc[i]))
reestr_mini['тип'] = swap_type

In [ ]:
# Своп-разница по сделке
swap_diff = []
for i in range(len(reestr_mini['№ сделки'])):
    swap_diff.append(reestr_mini['Курс2'].iloc[i] - reestr_mini['Курс1'].iloc[i])
reestr_mini['Своп-разница по сделке'] = swap_diff


In [ ]:
# Своп-разница мин direct swappoints
swp_min = []
for i in range(len(reestr_mini['№ сделки'])):
    table_name = 'swap_points_fwd'
    source = 'MOEX'
    pair_swap_name = reestr_mini['Валютная пара'].iloc[i][:3] + "_TODTOM"
    date = pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/ изменения условий'].iloc[i], dayfirst=True)
    with closing(sqlite3.connect('Data Archive.db')) as connection:
        result = connection.cursor().execute(f"""select low from '{table_name}' where source = '{source}' and name = '{pair_swap_name}' and date = '{date}' """).fetchall()
        swp_min.append(result[0][0] if result else None)

In [ ]:
# Своп-разница макс
swp_max = []
for i in range(len(reestr_mini['№ сделки'])):
    table_name = 'swap_points_fwd'
    source = 'MOEX'
    pair_swap_name = reestr_mini['Валютная пара'].iloc[i][:3] + "_TODTOM"
    date = pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/ изменения условий'].iloc[i], dayfirst=True)
    with closing(sqlite3.connect('Data Archive.db')) as connection:
        result = connection.cursor().execute(f"""select high from '{table_name}' where source = '{source}' and name = '{pair_swap_name}' and date = '{date}' """).fetchall()
        swp_max.append(result[0][0] if result else None)

In [ ]:
# Определяем валюту

cur_1 = []
for i in range(len(reestr_mini['№ сделки'])):
    cur_1.append(reestr_mini['Валютная пара'].iloc[i][3:])
reestr_mini['Валюта 1'] = cur_1

cur_2 = []
for i in range(len(reestr_mini['№ сделки'])):
    cur_2.append(reestr_mini['Валютная пара'].iloc[i][:3])
reestr_mini['Валюта 2'] = cur_2

In [ ]:
# Определяем тенор

tenor_low = []
for i in range(len(reestr_mini['№ сделки'])):
    tenor_low.append(tenor2l(reestr_mini['Срок'].iloc[i]))
reestr_mini['Тенор мин'] = tenor_low

tenor_high = []
for i in range(len(reestr_mini['№ сделки'])):
    tenor_high.append(tenor1h(reestr_mini['Срок'].iloc[i]))
reestr_mini['Тенор макс'] = tenor_high

In [ ]:
# Идентификаторы кривых ставок
curve_id_1 = []
for i in range(len(reestr_mini['№ сделки'])):
    curve_id_1.append(curve_ident(reestr_mini['Продаваемая Банком валюта'].iloc[i]))
reestr_mini['ставка_1'] = curve_id_1

curve_id_2 = []
for i in range(len(reestr_mini['№ сделки'])):
    curve_id_2.append(curve_ident(reestr_mini['Покупаемая Банком валюта'].iloc[i]))
reestr_mini['ставка_2'] = curve_id_2

In [ ]:
# Вспомогательные строки для ставок (parsing)
rates_domestic_l = []
for i in range(len(reestr_mini['№ сделки'])):
    rates_domestic_l.append(rate_domestic_pars_l(reestr_mini['Валюта 1'].iloc[i], pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/ изменения условий'].iloc[i]), reestr_mini['Тенор мин'].iloc[i], reestr_mini['ставка_1'].iloc[i]))
reestr_mini['Ставка внутреняя мин'] = rates_domestic_l

rates_domestic_h = []
for i in range(len(reestr_mini['№ сделки'])):
    rates_domestic_h.append(rate_domestic_pars_h(reestr_mini['Валюта 1'].iloc[i], pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/ изменения условий'].iloc[i]), reestr_mini['Тенор макс'].iloc[i], reestr_mini['ставка_1'].iloc[i]))
reestr_mini['Ставка внутреняя макс'] = rates_domestic_h

rates_foreign_l = []
for i in range(len(reestr_mini['№ сделки'])):
    rates_foreign_l.append(rate_foreign_pars_l(reestr_mini['Валюта 2'].iloc[i], pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/ изменения условий'].iloc[i]), reestr_mini['Тенор мин'].iloc[i], reestr_mini['ставка_2'].iloc[i]))
reestr_mini['Ставка внешняя мин'] = rates_foreign_l

rates_foreign_h = []
for i in range(len(reestr_mini['№ сделки'])):
    rates_foreign_h.append(rate_foreign_pars_h(reestr_mini['Валюта 2'].iloc[i], pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/ изменения условий'].iloc[i]), reestr_mini['Тенор макс'].iloc[i], reestr_mini['ставка_2'].iloc[i]))
reestr_mini['Ставка внешняя макс'] = rates_foreign_h

In [ ]:
# Интерполяция ставок
reestr_mini['Интер ставка внутреняя'] = reestr_mini.apply(lambda row: np.interp(row['Срок'], [row['Тенор мин'], row['Тенор макс']], [row['Ставка внутреняя мин'], row['Ставка внутреняя макс']]), axis=1)
reestr_mini['Интер ставка внешняя'] = reestr_mini.apply(lambda row: np.interp(row['Срок'], [row['Тенор мин'], row['Тенор макс']], [row['Ставка внешняя мин'], row['Ставка внешняя макс']]), axis=1)

In [ ]:
# Своп-поинты через ставки
reestr_mini['Своп-поинты мин_ставка'] = reestr_mini.apply(lambda row: ((row['Спот мин'] * ((1 + row['Интер ставка внутреняя']/100) / (1 + row['Интер ставка внешняя']/100)) ** (row['Срок']/365)) - row['Спот мин']), axis=1)
reestr_mini['Своп-поинты макс_ставка'] = reestr_mini.apply(lambda row: ((row['Спот макс'] * ((1 + row['Интер ставка внутреняя']/100) / (1 + row['Интер ставка внешняя']/100)) ** (row['Срок']/365)) - row['Спот макс']), axis=1)

In [ ]:
# Развилка по своп поинтам
swap_min_real = []
swap_max_real = []
for i in range(len(reestr_mini['№ сделки'])):
    if reestr_mini['тип'].iloc[i] == 'direct swp':
        swap_min_real.append(swp_min[i])
        swap_max_real.append(swp_max[i])
    else:
        swap_min_real.append(reestr_mini['Своп-поинты мин_ставка'].iloc[i])
        swap_max_real.append(reestr_mini['Своп-поинты макс_ставка'].iloc[i])

reestr_mini['swap min'] = swap_min_real
reestr_mini['swap max'] = swap_max_real

In [ ]:
# Расхождение своп
diff_swap = []
for i in range(len(reestr_mini['№ сделки'])):
    diff_swap.append(diff_swap_points(reestr_mini['Своп-разница по сделке'].iloc[i], reestr_mini['swap min'].iloc[i], reestr_mini['swap max'].iloc[i]))
reestr_mini['Расхождение своп'] = diff_swap

In [ ]:
# % расхождение в %
swap_points_percent = []
for i in range(len(reestr_mini['№ сделки'])):
    if reestr_mini['Расхождение своп'].iloc[i] == 'В рынке':
        swap_points_percent.append('-')
    else:
        try:
            swap_points_percent.append(reestr_mini['Расхождение своп'].iloc[i] / reestr_mini['Спот макс'].iloc[i])
        except (TypeError, ZeroDivisionError):
            swap_points_percent.append('-')
reestr_mini['Расхождение своп %'] = swap_points_percent

In [ ]:
# Discount factors
reestr_mini['DF_1'] = 1 / (1 + (reestr_mini['Интер ставка внутреняя'] * reestr_mini['Срок'] / 365))
reestr_mini['DF_2'] = 1 / (1 + (reestr_mini['Интер ставка внешняя'] * reestr_mini['Срок'] / 365))

In [ ]:
# Вспомогательная часть для курсов для СС
val_1l = []
for i in range(len(reestr_mini['№ сделки'])):
    if reestr_mini['Валютная пара 1'].iloc[i] == 'RUBRUB':
        val_1l.append(1)
    else:
        val_1l.append(reestr_mini['Спот мин val date'].iloc[i])
reestr_mini['Валютная пара 1, мин на дату оценки'] = val_1l

val_1h = []
for i in range(len(reestr_mini['№ сделки'])):
    if reestr_mini['Валютная пара 1'].iloc[i] == 'RUBRUB':
        val_1h.append(1)
    else:
        val_1h.append(reestr_mini['Спот макс val date'].iloc[i])
reestr_mini['Валютная пара 1, макс на дату оценки'] = val_1h

val_2l = []
for i in range(len(reestr_mini['№ сделки'])):
    if reestr_mini['Валютная пара 2'].iloc[i] == 'RUBRUB':
        val_2l.append(1)
    else:
        val_2l.append(reestr_mini['Спот мин val date'].iloc[i])
reestr_mini['Валютная пара 2, мин на дату оценки'] = val_2l

val_2h = []
for i in range(len(reestr_mini['№ сделки'])):
    if reestr_mini['Валютная пара 2'].iloc[i] == 'RUBRUB':
        val_2h.append(1)
    else:
        val_2h.append(reestr_mini['Спот макс val date'].iloc[i])
reestr_mini['Валютная пара 2, макс на дату оценки'] = val_2h

In [ ]:
# Курс ЦБ
CB_RF = []
for i in range(len(reestr_mini['№ сделки'])):
    table_name = 'fx_rate'
    source = 'CBR'
    pair_spot_name = reestr_mini['Валютная пара'].iloc[i]
    date = pd.to_datetime(reestr_mini['Текущая дата, на которую рассчитывалась справедливая стоимость ПФИ'].iloc[i], dayfirst=True)
    with closing(sqlite3.connect('Data Archive.db')) as connection:
        result = connection.cursor().execute(f"""select px_clir from '{table_name}' where source = '{source}' and pair = '{pair_spot_name}' and date = '{date}' """).fetchall()
        CB_RF.append(result[0][0] if result else None)
reestr_mini['курс ЦБ'] = CB_RF

In [ ]:
# Курсы для фв
cbr_1 = []
for i in range(len(reestr_mini['№ сделки'])):
    if reestr_mini['Валютная пара 1'].iloc[i] == 'RUBRUB':
        cbr_1.append(1)
    else:
        cbr_1.append(reestr_mini['курс ЦБ'].iloc[i])
reestr_mini['cbr_1'] = cbr_1

cbr_2 = []
for i in range(len(reestr_mini['№ сделки'])):
    if reestr_mini['Валютная пара 2'].iloc[i] == 'RUBRUB':
        cbr_2.append(1)
    else:
        cbr_2.append(reestr_mini['курс ЦБ'].iloc[i])
reestr_mini['cbr_2'] = cbr_2

In [ ]:
# Расчет справедливой стоимости
FVL_buy = []
FVL_sell = []
FVH_buy = []
FVH_sell = []
FVCB_buy = []
FVCB_sell = []
FVCRNCC_buy =[]
FVCRNCC_sell =[]


for i in range(len(reestr_mini['№ сделки'])):
    # Low buy
    FVL_buy.append((reestr_mini['Сумма покупки2'].iloc[i] * reestr_mini['DF_2'].iloc[i] * reestr_mini['Валютная пара 1, мин на дату оценки'].iloc[i] -
                   reestr_mini['Сумма продажи2'].iloc[i] * reestr_mini['DF_1'].iloc[i] * reestr_mini['Валютная пара 2, макс на дату оценки'].iloc[i]) / 10000)
    # Low sell
    FVL_sell.append((reestr_mini['Сумма покупки2'].iloc[i] * reestr_mini['DF_1'].iloc[i] * reestr_mini['Валютная пара 1, мин на дату оценки'].iloc[i] -
                    reestr_mini['Сумма продажи2'].iloc[i] * reestr_mini['DF_2'].iloc[i] * reestr_mini['Валютная пара 2, макс на дату оценки'].iloc[i]) / 10000)
    # High buy
    FVH_buy.append((reestr_mini['Сумма покупки2'].iloc[i] * reestr_mini['DF_2'].iloc[i] * reestr_mini['Валютная пара 1, макс на дату оценки'].iloc[i] -
                   reestr_mini['Сумма продажи2'].iloc[i] * reestr_mini['DF_1'].iloc[i] * reestr_mini['Валютная пара 2, мин на дату оценки'].iloc[i]) / 10000)
    # High sell
    FVH_sell.append((reestr_mini['Сумма покупки2'].iloc[i] * reestr_mini['DF_1'].iloc[i] * reestr_mini['Валютная пара 1, макс на дату оценки'].iloc[i] -
                    reestr_mini['Сумма продажи2'].iloc[i] * reestr_mini['DF_2'].iloc[i] * reestr_mini['Валютная пара 2, мин на дату оценки'].iloc[i]) / 10000)
    # CB buy
    FVCB_buy.append((reestr_mini['Сумма покупки2'].iloc[i] * reestr_mini['DF_2'].iloc[i] * reestr_mini['cbr_1'].iloc[i] -
                    reestr_mini['Сумма продажи2'].iloc[i] * reestr_mini['DF_1'].iloc[i] * reestr_mini['cbr_2'].iloc[i]) / 10000)
    # CB sell
    FVCB_sell.append((reestr_mini['Сумма покупки2'].iloc[i] * reestr_mini['DF_1'].iloc[i] * reestr_mini['cbr_1'].iloc[i] -
                     reestr_mini['Сумма продажи2'].iloc[i] * reestr_mini['DF_2'].iloc[i] * reestr_mini['cbr_2'].iloc[i]) / 10000)
    #НКЦ buy
    FVCRNCC_buy.append((reestr_mini['Сумма покупки2'].iloc[i] * reestr_mini['DF_2'].iloc[i] * reestr_mini['ncc_1'].iloc[i] -
                    reestr_mini['Сумма продажи2'].iloc[i] * reestr_mini['DF_1'].iloc[i] * reestr_mini['ncc_2'].iloc[i]) / 10000)
    #НКЦ sell
    FVCRNCC_sell.append((reestr_mini['Сумма покупки2'].iloc[i] * reestr_mini['DF_1'].iloc[i] * reestr_mini['ncc_1'].iloc[i] -
                     reestr_mini['Сумма продажи2'].iloc[i] * reestr_mini['DF_2'].iloc[i] * reestr_mini['ncc_2'].iloc[i]) / 10000)


FV_low = []
FV_high = []
FV_cb = []
FV_ncc =[]

for i in range(len(reestr_mini['№ сделки'])):
    if reestr_mini['Направление сделки'].iloc[i] == 'BUY':
        FV_low.append(FVL_buy[i])
        FV_high.append(FVH_buy[i])
        FV_cb.append(FVCB_buy[i])
        FV_ncc.append(FVCRNCC_buy[i])
    else:
        FV_low.append(FVL_sell[i])
        FV_high.append(FVH_sell[i])
        FV_cb.append(FVCB_sell[i])
        FV_ncc.append(FVCRNCC_sell[i])

reestr_mini['Справедливая стоимость на дату оценки low'] = FV_low
reestr_mini['Справедливая стоимость на дату оценки high'] = FV_high
reestr_mini['Справедливая стоимость на дату оценки по курсу ЦБ'] = FV_cb
reestr_mini['Справедливая стоимость на дату оценки по курсу ЦК НКЦ'] = FV_ncc


In [ ]:
# Расхождение с СС Банка
fv_delta_abs = []
fv_delta_percent = []

for i in range(len(reestr_mini['№ сделки'])):
    bank_fv = reestr_mini['Справедливая стоимость на текущую дату, в тыс. руб. (Только для ПФИ)'].iloc[i]
    low_fv = reestr_mini['Справедливая стоимость на дату оценки low'].iloc[i]
    high_fv = reestr_mini['Справедливая стоимость на дату оценки high'].iloc[i]

    if bank_fv < low_fv:
        fv_delta_abs.append(bank_fv - low_fv)
        fv_delta_percent.append((bank_fv - low_fv) * 1000 / reestr_mini['Сумма в расчетной валюте'].iloc[i])
    elif bank_fv > high_fv:
        fv_delta_abs.append(bank_fv - high_fv)
        fv_delta_percent.append((bank_fv - high_fv) * 1000 / reestr_mini['Сумма в расчетной валюте'].iloc[i])
    else:
        fv_delta_abs.append("В рынке")
        fv_delta_percent.append(0.)

reestr_mini['Расхождение с СС Банка (абс)'] = fv_delta_abs
reestr_mini['Расхождение с СС Банка (%)'] = fv_delta_percent

reestr_mini.head()

,№ сделки,№ сделки (идентификатор сделки),"Статус сделки (новая, пролонгация, изменение условий, закрыта и др.)","Дата сделки, дата пролонгации/ изменения условий",Контрагент,Продаваемая валюта1,Сумма продажи1,Покупаемая валюта1,Сумма покупки1,Курс1,...,"Валютная пара 2, макс на дату оценки",курс ЦБ,cbr_1,cbr_2,Справедливая стоимость на дату оценки low,Справедливая стоимость на дату оценки high,Справедливая стоимость на дату оценки по курсу ЦБ,Справедливая стоимость на дату оценки по курсу ЦК НКЦ,Расхождение с СС Банка (абс),Расхождение с СС Банка (%)
0,1,NaN,Новая,31.07.2024,NaN,RUB,162335547,CNY,13700000,11.84931,...,11.8288,11.8747,1,11.8747,271.586512,498.186953,211.604042,244.783887,-56.496512,-0.000348
1,2,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,...,11.8288,11.8747,1,11.8747,1.982383,3.636401,1.544555,1.786744,-0.412383,-0.000348
2,3,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,...,11.8288,11.8747,1,11.8747,1.982383,3.636401,1.544555,1.786744,-0.412383,-0.000348
3,4,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,...,11.8288,11.8747,1,11.8747,1.982383,3.636401,1.544555,1.786744,-0.412383,-0.000348
4,5,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,...,11.8288,11.8747,1,11.8747,1.982383,3.636401,1.544555,1.786744,-0.412383,-0.000348


In [ ]:
reestr_mini.to_excel('full_calculations_of_reestr.xlsx')

# FINAL REESTR

In [ ]:
reestr_initial = pd.read_excel('reestr_new_1.xlsx')

In [ ]:
reestr_initial['Продаваемая банком валюта'] = reestr_mini['Продаваемая Банком валюта']
reestr_initial['Покупаемая банком валюта'] = reestr_mini['Покупаемая Банком валюта']
reestr_initial['Валютная пара'] = reestr_mini['Валютная пара']
reestr_initial['Направление сделки (первой части сделки)'] = reestr_mini['Направление сделки']
reestr_initial['Название'] = reestr_mini['Название']
reestr_initial['Сумма в базовой валюте'] = reestr_mini['Сумма в базовой валюте']
reestr_initial['Сумма в расчетной валюте'] = reestr_mini['Сумма в расчетной валюте']
reestr_initial['Срок'] = reestr_mini['Срок']
reestr_initial['Курс спот сделки'] = reestr_mini['Курс спот сделки (1 части)']
reestr_initial['ЦК НКЦ'] = reestr_mini['ЦК НКЦ']
reestr_initial['Спот курс min'] = reestr_mini['Спот мин']
reestr_initial['Спот курс max'] = reestr_mini['Спот макс']
reestr_initial['Расхождение'] = reestr_mini['Расхождение']
reestr_initial['Своп-разница по сделке'] = reestr_mini['Своп-разница по сделке']
reestr_initial['Своп-разница min'] = reestr_mini['swap min']
reestr_initial['Своп-разница max'] = reestr_mini['swap max']
reestr_initial['Расхождение'] = reestr_mini['Расхождение своп']
reestr_initial['Справедливая стоимость Банка'] = reestr_mini['Справедливая стоимость на текущую дату, в тыс. руб. (Только для ПФИ)']
reestr_initial['Справедливая стоимость Службы, LOW'] = reestr_mini['Справедливая стоимость на дату оценки low']
reestr_initial['Справедливая стоимость Службы, HIGH'] = reestr_mini['Справедливая стоимость на дату оценки high']
reestr_initial['Справедливая стоимость на дату оценки по курсу ЦБ'] = reestr_mini['Справедливая стоимость на дату оценки по курсу ЦБ']
reestr_initial['Справедливая стоимость на дату оценки по курсу ЦК НКЦ'] = reestr_mini['Справедливая стоимость на дату оценки по курсу ЦК НКЦ']
reestr_initial['Дата оценки'] = reestr_mini['Текущая дата, на которую рассчитывалась справедливая стоимость ПФИ']
reestr_initial["Расхождение % относительно номинала сделки"] = fv_delta_percent
reestr_initial["Расхождение abs, тыс. руб."] = fv_delta_abs
reestr_initial['Расхождение с СС Банка (%)'] = fv_delta_percent

reestr_initial.to_excel('reestr_fianl.xlsx')

reestr_initial.head()

,№ сделки,№ сделки (идентификатор сделки),"Статус сделки (новая, пролонгация, изменение условий, закрыта и др.)","Дата сделки, дата пролонгации/ изменения условий",Контрагент,Продаваемая валюта1,Сумма продажи1,Покупаемая валюта1,Сумма покупки1,Курс1,...,Своп-разница max,Справедливая стоимость Банка,"Справедливая стоимость Службы, LOW","Справедливая стоимость Службы, HIGH",Справедливая стоимость на дату оценки по курсу ЦБ,Справедливая стоимость на дату оценки по курсу ЦК НКЦ,Дата оценки,Расхождение % относительно номинала сделки,"Расхождение abs, тыс. руб.",Расхождение с СС Банка (%)
0,1,NaN,Новая,31.07.2024,NaN,RUB,162335547,CNY,13700000,11.84931,...,0.00559,215.09,271.586512,498.186953,211.604042,244.783887,01.08.2024,-0.000348,-56.496512,-0.000348
1,2,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,...,0.00559,1.57,1.982383,3.636401,1.544555,1.786744,01.08.2024,-0.000348,-0.412383,-0.000348
2,3,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,...,0.00559,1.57,1.982383,3.636401,1.544555,1.786744,01.08.2024,-0.000348,-0.412383,-0.000348
3,4,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,...,0.00559,1.57,1.982383,3.636401,1.544555,1.786744,01.08.2024,-0.000348,-0.412383,-0.000348
4,5,NaN,Новая,31.07.2024,NaN,RUB,1184931,CNY,100000,11.84931,...,0.00559,1.57,1.982383,3.636401,1.544555,1.786744,01.08.2024,-0.000348,-0.412383,-0.000348


In [ ]:
reestr_initial.to_excel('final_reestr.xlsx')

# **Design part**

In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
import xlsxwriter
import datetime

with pd.ExcelWriter('final_swap_with_design.xlsx', engine='xlsxwriter', engine_kwargs={'options': {'nan_inf_to_errors': True}}) as wb:

    data_border_format = wb.book.add_format({
        'border': 1,
        'align': 'center',
        'valign': 'vcenter'
    })

    date_format = wb.book.add_format({
        'border': 1,
        'align': 'center',
        'valign': 'vcenter',
        'num_format': 'dd.mm.yyyy'
    })


    number_format = wb.book.add_format({
        'border': 1,
        'align': 'center',
        'valign': 'vcenter',
        'num_format': '### ### ###'
    })

    reestr_initial.to_excel(wb, sheet_name='СПОТ', startrow=7, startcol=1, index=False)

    sheet = wb.sheets['СПОТ']

    sheet.hide_gridlines(2)

    title_format = wb.book.add_format({
        'bold' : True,
        'align': 'left',
        'valign': 'vcenter'})

    sheet.write(6, 1, "Валютный своп", title_format)
    sheet.write(2, 17, "Для служебного пользования")
    sheet.write(3, 17, "Приложение 3 к письму")

    num_rows = len(reestr_initial)
    num_cols = len(reestr_initial.columns)

    number_columns = [7, 9, 13, 15, 26, 27]
    date_columns = [4, 11, 17, 20, 42]


    for row in range(8, 8 + num_rows):
        for col in range(1, 1 + num_cols):
            value = reestr_initial.iloc[row-8, col-1]


            if col in date_columns:

                if pd.isna(value) or value in [float('inf'), float('-inf')]:
                    sheet.write(row, col, "", date_format)
                else:

                    if isinstance(value, (pd.Timestamp, datetime.datetime)):
                        sheet.write_datetime(row, col, value, date_format)
                    else:
                        sheet.write(row, col, value, date_format)


            elif col in number_columns:

                if pd.isna(value) or value in [float('inf'), float('-inf')]:
                    sheet.write(row, col, "", number_format)
                else:

                    if isinstance(value, (int, float)) and not pd.isna(value):
                        sheet.write_number(row, col, value, number_format)
                    else:
                        sheet.write(row, col, value, number_format)

            else:

                if pd.isna(value) or value in [float('inf'), float('-inf')]:
                    sheet.write(row, col, "", data_border_format)
                else:
                    sheet.write(row, col, value, data_border_format)


    gray_format = wb.book.add_format({
        'bold': True,
        'font_color': '#000000',
        'bg_color': '#DCDCDC',  ##DCDCDC, c0c0c0 можно сделать еще посветлее серый
        'align': 'center',
        'valign': 'vcenter',
        'border': 1})

    blue_format = wb.book.add_format({
        'bold': True,
        'font_color': '#FFFFFF',
        'bg_color': '#0066cc',
        'align': 'center',
        'valign': 'vcenter',
        'border': 1})



    black_format = wb.book.add_format({
        'bold': True,
        'font_color': '#FFFFFF',
        'bg_color': '#404040',
        'align': 'center',
        'valign': 'vcenter',
        'border': 1})


    for col_idx, col_name in enumerate(reestr_initial.columns):
        if col_idx < 28:
            sheet.write(7, col_idx + 1, col_name, gray_format)
        elif col_idx < 33:
            sheet.write(7, col_idx + 1, col_name, blue_format)
        elif col_idx < 45:
            sheet.write(7, col_idx + 1, col_name, black_format)
        # else:
        #     sheet.write(7, col_idx + 1, col_name, gray_format)


    for i, col in enumerate(reestr_initial.columns):
        max_len = 0
        for row in reestr_initial[col]:
            try:

                if pd.isna(row) or row in [float('inf'), float('-inf')]:
                    cell_value = ""
                else:

                    if i in [2, 3] and isinstance(row, (pd.Timestamp, datetime.datetime)):
                        cell_value = row.strftime('%d.%m.%Y')
                    else:
                        cell_value = str(row)
                max_len = max(max_len, len(cell_value))
            except:
                pass
        max_len = max(max_len, len(col))
        sheet.set_column(i + 1, i + 1, max_len + 2)

    for i in range(len(reestr_initial) + 1):
        sheet.set_row(i + 7, None)